In [33]:
import numpy as np
import pandas as pd
import copy
import random

In [34]:
class GA_Sudoku_Solver:
    def __init__(self, archivo, popsize=400, NGenerations=1000, probCrossover=0.7, probMutation=0.4):
        #Cargar archivo
        sudoku_df = pd.read_csv(archivo, header=None)
        self.sudoku_matrix = sudoku_df.values        
        #Parámetros del algoritmo genético
        self.popsize = popsize
        self.NGenerations = NGenerations
        self.probCrossover = probCrossover
        self.probMutation = probMutation
        #Inicialización de la población y variables auxiliares
        self.population = [None] * popsize
        self.fitness = np.zeros((self.popsize), int)
        self.elite = None
        self.eliteFit = None
    
    def init_population(self):
        allValues = set(range(1, 10)) 

        # Crear lista de valores faltantes por fila
        missingValues = [list(allValues - set(self.sudoku_matrix[i])) for i in range(9)]
        
        #Crear población inicial
        for i in range(self.popsize):
            newInd = copy.deepcopy(missingValues)
            for j in range(9):
                random.shuffle(newInd[j])  #Barajar los valores faltantes
            self.population[i] = newInd
            self.fitness[i] = self.fitness_individual(newInd)

    #Selecciona 2 individuos al azar y regresa el mejor
    def tournament_selection(self):
        i1 = np.random.randint(self.popsize)
        i2 = np.random.randint(self.popsize)
        if self.fitness[i1] <= self.fitness[i2]:
            return i1
        else:
            return i2

    def crossover(self, par1, par2):
        offspring = [None] * 9
        offspring[0:4] = copy.deepcopy(par1[0:4])  # Primeras 4 filas del primer padre
        offspring[4:9] = copy.deepcopy(par2[4:9])  # Últimas 5 filas del segundo padre
        return offspring
    
    #Intercambia valores dentro de una fila
    def mutation(self,ind):
        row = np.random.randint(9)
        while True:
            i1 = np.random.randint(len(ind[row]))
            i2 = np.random.randint(len(ind[row]))
            if i1!=i2: break
        ind[row][i1],ind[row][i2] = ind[row][i2],ind[row][i1]
        return ind

    #Calcula la aptitud de un individuo contando los conflictos en columnas y subcuadrículas.
    def fitness_individual(self, ind):
        errors = 0
        board = self.create_board(ind)

        #Contar conflictos en columnas
        for i in range(9):
            column = [board[j][i] for j in range(9)]
            errors += 9 - len(set(column))
        
        #Contar conflictos en subcuadrículas 3x3
        for i in range(0, 9, 3):
            for j in range(0, 9, 3):
                subgrid = [board[x][y] for x in range(i, i+3) for y in range(j, j+3)]
                errors += 9 - len(set(subgrid))
        
        return errors

    def create_board(self, ind):
        board = copy.deepcopy(self.sudoku_matrix)
        for i in range(9):
            indx = 0
            for j in range(9):
                if board[i][j] == 0:  #Rellena las celdas vacías
                    board[i][j] = ind[i][indx]
                    indx += 1
        return board
    
    def update_elite(self):
        i = np.argmin(self.fitness)
        if self.eliteFit is None or self.fitness[i] < self.eliteFit:
            self.elite = copy.deepcopy(self.population[i])
            self.eliteFit = self.fitness[i]
        else:
            # Reemplaza un individuo aleatorio con el mejor (elite) si no hay mejora
            i = np.random.randint(self.popsize)
            self.population[i] = self.elite
            self.fitness[i] = self.eliteFit

    def exec(self):
        self.init_population()
        self.update_elite()

        for k in range(self.NGenerations):
            if self.eliteFit == 0:  # Termina si se encuentra una solución perfecta
                break

            offspring = [None] * self.popsize
            offspringFit = np.zeros((self.popsize), int)

            # Generación de la nueva población
            for i in range(self.popsize):
                i1 = self.tournament_selection()
                i2 = self.tournament_selection()
                if np.random.rand() <= self.probCrossover:
                    of = self.crossover(self.population[i1], self.population[i2])
                else:
                    of = copy.deepcopy(self.population[i1])
                if np.random.rand() <= self.probMutation:
                    of = self.mutation(of)
                offspring[i] = of
                offspringFit[i] = self.fitness_individual(of)
            
            self.population = offspring
            self.fitness = offspringFit
            self.update_elite()

            #Imprimir progreso de la generación
            print(f'Generacion {k + 1} \terrores: {self.eliteFit}')
        
        #Imprimir la mejor solución encontrada
        print(f"Mejor solución encontrada: Generacion {k + 1} errores: {self.eliteFit}")
        print(self.create_board(self.elite))



In [35]:
#Sudoku 1
# Ejecución del algoritmo genético para resolver el Sudoku
ga = GA_Sudoku_Solver('sudoku1.csv')
ga.exec()


Generacion 1 	errores: 16
Generacion 2 	errores: 16
Generacion 3 	errores: 14
Generacion 4 	errores: 11
Generacion 5 	errores: 10
Generacion 6 	errores: 9
Generacion 7 	errores: 8
Generacion 8 	errores: 8
Generacion 9 	errores: 8
Generacion 10 	errores: 6
Generacion 11 	errores: 4
Generacion 12 	errores: 4
Generacion 13 	errores: 4
Generacion 14 	errores: 2
Generacion 15 	errores: 2
Generacion 16 	errores: 2
Generacion 17 	errores: 2
Generacion 18 	errores: 2
Generacion 19 	errores: 0
Mejor solución encontrada: Generacion 20 errores: 0
[[9 6 1 7 4 3 5 8 2]
 [7 2 3 6 8 5 9 4 1]
 [8 4 5 9 2 1 6 7 3]
 [3 9 6 1 7 8 4 2 5]
 [1 7 4 5 9 2 3 6 8]
 [5 8 2 4 3 6 1 9 7]
 [4 3 7 8 1 9 2 5 6]
 [6 1 9 2 5 7 8 3 4]
 [2 5 8 3 6 4 7 1 9]]


In [37]:
#Sudoku 2
ga = GA_Sudoku_Solver('sudoku2.csv')
ga.exec()


Generacion 1 	errores: 26
Generacion 2 	errores: 24
Generacion 3 	errores: 24
Generacion 4 	errores: 24
Generacion 5 	errores: 21
Generacion 6 	errores: 21
Generacion 7 	errores: 21
Generacion 8 	errores: 21
Generacion 9 	errores: 20
Generacion 10 	errores: 18
Generacion 11 	errores: 18
Generacion 12 	errores: 17
Generacion 13 	errores: 15
Generacion 14 	errores: 15
Generacion 15 	errores: 15
Generacion 16 	errores: 15
Generacion 17 	errores: 15
Generacion 18 	errores: 15
Generacion 19 	errores: 14
Generacion 20 	errores: 13
Generacion 21 	errores: 11
Generacion 22 	errores: 11
Generacion 23 	errores: 10
Generacion 24 	errores: 10
Generacion 25 	errores: 10
Generacion 26 	errores: 8
Generacion 27 	errores: 8
Generacion 28 	errores: 8
Generacion 29 	errores: 6
Generacion 30 	errores: 6
Generacion 31 	errores: 6
Generacion 32 	errores: 6
Generacion 33 	errores: 6
Generacion 34 	errores: 6
Generacion 35 	errores: 6
Generacion 36 	errores: 6
Generacion 37 	errores: 6
Generacion 38 	errores